In [1]:
import pandas as pd
import numpy as np
import io
import struct
import os
from PIL import Image
from matplotlib import pyplot as plt
from skimage.io import imread, imshow
from skimage.transform import resize, rotate
import shutil
import boto3
import glob
import file_image_processor

In [12]:
s3client = boto3.client('s3')

In [43]:
fp = "/home/newton/images/gender_images/train/male"
attr_list = list()
for root, dirs, files in os.walk(fp, topdown=False):
    for f in files:
        attr_list.append(getImData(os.path.join(fp,f)))
        
        

In [44]:
n1 = np.array(attr_list)
n1[:4]

array([[178, 218],
       [178, 218],
       [178, 218],
       [178, 218]])

In [45]:
print(np.unique(n1))

[178 218]


In [2]:
anno = pd.read_csv('./Anno/list_attr_celeba.txt', header=0, delimiter=r"\s+", error_bad_lines=False)

FileNotFoundError: [Errno 2] File b'./Anno/list_attr_celeba.txt' does not exist: b'./Anno/list_attr_celeba.txt'

In [3]:
nbindx = anno.columns.get_loc('No_Beard')
gtindx = anno.columns.get_loc('Goatee')
focindx = anno.columns.get_loc('5_o_Clock_Shadow')
muindx = anno.columns.get_loc('Mustache')
maindx = anno.columns.get_loc('Male')


In [5]:
anno['face_hair'] = -1
anno['no_face_hair'] = -1
fhindx = anno.columns.get_loc('face_hair')
nfhindx = anno.columns.get_loc('no_face_hair')


In [6]:
def check_no_face_hair(place):
    
    if anno.iloc[place,maindx] == 1:
        if (anno.iloc[place,nbindx] == 1 and anno.iloc[place,gtindx] == -1 and
            anno.iloc[place,focindx] == -1 and anno.iloc[place,muindx] == -1):

            anno.iloc[place,nfhindx] = 1
    
                                                                       

In [7]:
def check_face_hair(place):
    if anno.iloc[place,maindx] == 1:
        if (anno.iloc[place,nbindx] == -1 or anno.iloc[place,gtindx] == 1 or
            anno.iloc[place,muindx] == 1):

            anno.iloc[place,fhindx] = 1
        

In [8]:
for i in range(anno.shape[0]):
    check_face_hair(i)
    check_no_face_hair(i)

In [13]:
def train_data_maker(data_folder, num_images, c_name, id, max_images):
    ''' data_folder    -   the path to write the imags to
        num_images     -   the number of images to write
        c_name         -   the name of the column
        id             -   the value to use             
        max_images     -   the maximum number of images to look through '''

    count = 0
    cindx = anno.columns.get_loc(c_name)
   
    for i in range(max_images):
        filename = anno.iloc[i,0]
        if count == num_images:
            break
        
        if anno.iloc[i,cindx] == id:
            s3client.download_file('fakefacemaker',filename, os.path.join(data_folder, filename))
            count += 1

In [14]:
train_data_maker('/home/newton/images/facehair_train/nohair', 4000, 'no_face_hair', 1, 100000)

In [15]:
train_data_maker('/home/newton/images/facehair_train/hair', 4000, 'face_hair', 1, 150000)

In [24]:
# Creates the train a val folders in preparation for training a model.
classes = ['hair', 'nohair']
base_dir= '/home/newton/images/facehair_train'
for cl in classes:
  img_path = os.path.join(base_dir, cl)
  images = glob.glob(img_path + '/*.jpg')
  print("{}: {} Images".format(cl, len(images)))
  num_train = int(round(len(images)*0.8))
  train, val = images[:num_train], images[num_train:]

  for t in train:
    if not os.path.exists(os.path.join(base_dir, 'train', cl)):
      os.makedirs(os.path.join(base_dir, 'train', cl))
    shutil.move(t, os.path.join(base_dir, 'train', cl))

  for v in val:
    if not os.path.exists(os.path.join(base_dir, 'val', cl)):
      os.makedirs(os.path.join(base_dir, 'val', cl))
    shutil.move(v, os.path.join(base_dir, 'val', cl))

hair: 4000 Images
nohair: 4000 Images


In [18]:
source = '/home/newton/images/facehair_train/hair'
dest = '/home/newton/images/working'

In [19]:
image_mover(source, dest)

In [20]:
image_resizer(source,dest,224,224)

In [21]:
source = '/home/newton/images/facehair_train/nohair'

In [22]:
image_mover(source, dest)

In [23]:
image_resizer(source,dest,224,224)